### 문장 전처리

문장 전처리를 위한 코드입니다. 차례대로 구현되어 있으므로 따라서 보시면 됩니다.

하나로 묶은 파일은 contexts_preprocess.py를 참고해주시면 됩니다.

In [ ]:
# 필요한 객체 import
import pandas as pd
from konlpy.tag import Okt
from preprocessing import preprocess_data, tokenizing_data, get_label
from utils import seed_fix
import pandas as pd
import numpy as np
import random
import pickle

import re
# 초기화 및 모델 학습
from gensim.models import word2vec
# 띄어쓰기
from pykospacing import Spacing
# 마춤뻡 검사기
from hanspell import spell_checker

from soynlp.normalizer import *

가장 먼저 nan 값 처리를 위한 조치입니다.

nan 값은 type이 float로 나오게 되어서 아래와 같은 식으로 조치를 해주었습니다

In [ ]:
for word in contexts:
    if type(word) == float:
        contexts.remove(word)

두번째로는 중복 문자에 대한 처리입니다.

두 글자 이상 반복되는 글자는 제거해주었습니다

- ex) ㅋㅋㅋㅋㅋ, ㅠㅜㅠㅠㅠ 등

In [ ]:
def remove_repeat_char(contexts):
    preprocessed_text = []
    for text in contexts:
        text = repeat_normalize(text, num_repeats=2).strip()
        if text:
            preprocessed_text.append(text)

    return preprocessed_text

ㅠ, ㅋ 과 같은 한 글자씩만 등장하는 단어 혹은 특수 문자를 모두 제거

In [ ]:
# 한국어 정규식 확인 + ㅏ, ㅋ 이런 거 제거
def regular_check(contexts):
    preprocessed_text = []
    for text in contexts:
        text = re.sub('[ㄱ-ㅎㅏ-ㅣ]', '', text)
        text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', text)
        if text:
            preprocessed_text.append(text)

    return preprocessed_text

특수문자 통일

In [ ]:
def clean_punc(texts):
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

    preprocessed_text = []
    for text in texts:
        for p in punct_mapping:
            text = text.replace(p, punct_mapping[p])
        text = text.strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

띄어쓰기 1회만 되도록 + 양쪽 공백strip 처리

In [ ]:
def remove_repeated_spacing(texts):
    """
    두 개 이상의 연속된 공백을 하나로 치환합니다.
    ``오늘은    날씨가   좋다.`` -> ``오늘은 날씨가 좋다.``
    """
    preprocessed_text = []
    for text in texts:
        text = re.sub(r"\s+", " ", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

spacing 라이브러리를 통한 띄어쓰기

In [ ]:
def spacing_sent(texts):
    """
    띄어쓰기를 보정합니다.
    """
    preprocessed_text = []
    spacing = Spacing()
    for text in texts:
        text = spacing(text)
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

마춤뻡 검사기를 통한 처리

In [ ]:
def spell_check_sent(texts):
    """
    맞춤법을 보정합니다.
    """
    preprocessed_text = []
    for text in texts:
        try:
            spelled_sent = spell_checker.check(text)
            checked_sent = spelled_sent.checked 
            if checked_sent:
                preprocessed_text.append(checked_sent)
        except:
            preprocessed_text.append(text)
    return preprocessed_text